In [25]:
from os import walk, path
from itertools import combinations, product
from subprocess import Popen, PIPE

In [8]:
russian_models = []

for a, b, c in walk(path.join('..', 'monolang', 'MODELS', 'Word2Vec', 'russian')):
    for q in c:
        russian_models.append(path.join(a, q))

In [9]:
english_models = []

for a, b, c in walk(path.join('..', 'monolang', 'MODELS', 'Word2Vec', 'english')):
    for q in c:
        english_models.append(path.join(a, q))

In [14]:
combs = list(product(russian_models, english_models))

In [40]:
def remove_pos(model):
    model_vocab = set()
    new_model = []
    # new_model.append(model[0])
    for line in model[1:~0]:
        word, vector = line.split(' ', 1)
        word = word.split('_')[0]
        if word not in model_vocab:
            model_vocab.add(word)
            new_model.append('{} {}\n'.format(word, vector))
    vocab_size = len(model_vocab)
    vector_size = model[0].split(' ')[1]
    return ['{} {}\n'.format(vocab_size, vector_size)] + new_model

In [41]:
def create_model_for_training(model_path, model_lang):
    with open(model_path, 'r') as f:
        model = f.read().split('\n')
    new_model = remove_pos(model)
    with open(path.join('..', 'cross-lang', 'MUSE', '{}.vec'.format(model_lang)), 'w') as f:
        for line in new_model:
            f.write(line)

In [ ]:
for model_1, model_2 in combs[:1]:
    ru_name, en_name = model_1.split('/')[~0].split('.')[0], model_2.split('/')[~0].split('.')[0]
    create_model_for_training(model_1, 'ru')
    create_model_for_training(model_1, 'en')
    bashCommand = 'python ../cross-lang/MUSE/supervised.py --src_lang en --tgt_lang ru --src_emb en.vec tgt_emb ru.vec --dico_train default --cuda 0 --exp_path /notebook/cross-lang'
    bashCommand += ' --exp_name ' + '{}-{}'.format(en_name, ru_name)
    process = Popen(bashCommand.split(), stdout=PIPE)
    output, error = process.communicate()